In [ ]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression



In [ ]:
df = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
df.head()

In [ ]:
def encodeing_df(df):
        col_name = []
        label_encoder = LabelEncoder()
        for (colname, colval) in df.iteritems():
            if colval.dtype == 'object'or colval.dtype == 'bool':
                col_name.append(colname)
        for col in col_name:
            df[col] = label_encoder.fit_transform(df[col])
        return df

In [ ]:
def replace_null(df):
        col_nan = []
        for (colname, colval) in df.iteritems():
            if df[colname].isnull().values.any() == True:
                col_nan.append(colname)
        for col in col_nan:
            mean_value = df[col].mean()
            df[col].fillna(value=mean_value, inplace=True)
        return df

In [ ]:
def scaling(df):
        x = df.iloc[:, :-1]  # Using all column except for the last column as X
        y = df.iloc[:, -1]  # Selecting the last
        m = x.max() - x.min()
        m = m.to_dict()
        l = []
        for key, val in m.items():
            if val == 0:
                l.append(key)
        for i in l:
            x = x.drop([i], axis=1)
        df_norm = (x-x.min())/(x.max()-x.min())
        df_norm = pd.concat((df_norm, y), 1)
        return df_norm

In [ ]:
def cal_mean(df):
        df_mean = df.mean(axis=0).mean(axis=0)
        return df_mean

In [ ]:
def cal_std(df):
        col_std = df.std()
        col = col_std.mean(axis=0)
        return col

In [ ]:
def pre_proses_data(df):
        df = encodeing_df(df)
        df = replace_null(df)
#         df=scaling(df)
        return df

In [ ]:
df=pre_proses_data(df)

In [ ]:
df.head(50)

In [ ]:
def split_data(df, split_size):
    X = df.iloc[:, :-1]  # Using all column except for the last column as X
    Y = df.iloc[:, -1]  # Selecting the last column as Y

    X_train, X_test, Y_train, Y_test = train_test_split(
        X, Y, test_size=(100-split_size)/100)

    return X_train, X_test, Y_train, Y_test

In [ ]:
(X_train, X_test, Y_train, Y_test) = split_data(df, 80)

In [ ]:
def rf_param_selector(criterion,n_estimators,max_depth,min_samples_split,max_features):

    params = {
        "criterion": criterion,
        "n_estimators": n_estimators,
        "max_depth": max_depth,
        "min_samples_split": min_samples_split,
        "max_features": max_features,
        "n_jobs": -1,
    }

    model = RandomForestClassifier(**params)
    return model


In [ ]:
model = rf_param_selector("gini", 70, 17, 3, "sqrt")

In [ ]:
def train_model(model, x_train, y_train, x_test, y_test):
    model.fit(x_train, y_train)
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    train_accuracy = np.round(accuracy_score(y_train, y_train_pred), 3)
    train_f1 = np.round(f1_score(y_train, y_train_pred, average="weighted"), 3)

    test_accuracy = np.round(accuracy_score(y_test, y_test_pred), 3)
    test_f1 = np.round(f1_score(y_test, y_test_pred, average="weighted"), 3)

    return model, train_accuracy, train_f1, test_accuracy, test_f1

In [ ]:
(model, train_accuracy, train_f1, test_accuracy, test_f1) = train_model(model, X_train, Y_train, X_test, Y_test)

In [186]:
print(test_accuracy)
print(train_accuracy)

0.801
0.939


In [ ]:
test = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")
test=pre_proses_data(test)
test

In [ ]:
hot_train = pd.get_dummies(df, columns=["HomePlanet","CryoSleep","Destination","VIP"], drop_first=True)
hot_test = pd.get_dummies(test, columns=["HomePlanet","CryoSleep","Destination","VIP"], drop_first=True)

# hot_train["Transported"].replace({True:1,False:0}, inplace=True)
x = hot_train.drop('Transported',axis=1)
y = hot_train['Transported']

pred = model.predict(X_test)
y_pred = model.predict(X_test)
pred = model.predict(test)

In [187]:
submission = pd.read_csv('/kaggle/input/spaceship-titanic/sample_submission.csv')
submission['Transported'] = pred
submission['Transported'].replace({1:True,0:False}, inplace=True)
submission.to_csv("submission.csv", index=False)
submission.head(10)

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,False
5,0027_01,True
6,0029_01,True
7,0032_01,True
8,0032_02,True
9,0033_01,True
